In [23]:
import yaml
import os
from dotenv import load_dotenv
load_dotenv()

os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

In [24]:
from crewai import Agent, Task, Crew,LLM
ollama_llm = LLM(
            model="ollama/llama3.2:latest",
            #model="ollama/deepseek-r1:latest",
            #model="ollama/llama3.1:latest",
            api_base="http://localhost:11434"
            )

In [25]:
# Define file paths for YAML configurations
files = {
    'lead_agents': 'config/lead_qualification_agents.yaml',
    'lead_tasks': 'config/lead_qualification_tasks.yaml',
    'email_agents': 'config/email_engagement_agents.yaml',
    'email_tasks': 'config/email_engagement_tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
lead_agents_config = configs['lead_agents']
lead_tasks_config = configs['lead_tasks']
email_agents_config = configs['email_agents']
email_tasks_config = configs['email_tasks']

## Create Pydantic Models for Structured Output

In [26]:
from pydantic import BaseModel, Field
from typing import Dict, Optional, List, Set, Tuple

class LeadPersonalInfo(BaseModel):
    name: str = Field(..., description="The full name of the lead.")
    job_title: str = Field(..., description="The job title of the lead.")
    role_relevance: int = Field(..., ge=0, le=10, description="A score representing how relevant the lead's role is to the decision-making process (0-10).")
    professional_background: Optional[str] = Field(..., description="A brief description of the lead's professional background.")

class CompanyInfo(BaseModel):
    company_name: str = Field(..., description="The name of the company the lead works for.")
    industry: str = Field(..., description="The industry in which the company operates.")
    company_size: int = Field(..., description="The size of the company in terms of employee count.")
    revenue: Optional[float] = Field(None, description="The annual revenue of the company, if available.")
    market_presence: int = Field(..., ge=0, le=10, description="A score representing the company's market presence (0-10).")

class LeadScore(BaseModel):
    score: int = Field(..., ge=0, le=100, description="The final score assigned to the lead (0-100).")
    scoring_criteria: List[str] = Field(..., description="The criteria used to determine the lead's score.")
    validation_notes: Optional[str] = Field(None, description="Any notes regarding the validation of the lead score.")

class LeadScoringResult(BaseModel):
    personal_info: LeadPersonalInfo = Field(..., description="Personal information about the lead.")
    company_info: CompanyInfo = Field(..., description="Information about the lead's company.")
    lead_score: LeadScore = Field(..., description="The calculated score and related information for the lead.")

### Importing Tools

In [27]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

## Lead Qualification Crew, Agents and Tasks

In [28]:
# Creating Agents
lead_data_agent = Agent(
  config=lead_agents_config['lead_data_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
  # llm= ollama_llm
)

cultural_fit_agent = Agent(
  config=lead_agents_config['cultural_fit_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
  #llm= ollama_llm
)

scoring_validation_agent = Agent(
  config=lead_agents_config['scoring_validation_agent'],
  tools=[SerperDevTool(), ScrapeWebsiteTool()],
  #llm= ollama_llm
)

# Creating Tasks
lead_data_task = Task(
  config=lead_tasks_config['lead_data_collection'],
  agent=lead_data_agent
)

cultural_fit_task = Task(
  config=lead_tasks_config['cultural_fit_analysis'],
  agent=cultural_fit_agent
)

scoring_validation_task = Task(
  config=lead_tasks_config['lead_scoring_and_validation'],
  agent=scoring_validation_agent,
  context=[lead_data_task, cultural_fit_task],
  output_pydantic=LeadScoringResult
)

In [29]:

# Creating Crew
lead_scoring_crew = Crew(
  agents=[
    lead_data_agent,
    cultural_fit_agent,
    scoring_validation_agent
  ],
  tasks=[
    lead_data_task,
    cultural_fit_task,
    scoring_validation_task
  ],
  verbose=True
)

2025-03-06 15:15:06,847 - 8687798336 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [30]:
"""
sample_input =  {
                "lead_data": {
                    "name": "João Moura",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "joao@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            }

test_crew_1 = Crew(
  agents=[lead_data_agent],
  tasks=[lead_data_task] )

sample_result_1 = test_crew_1.kickoff(sample_input)
"""

'\nsample_input =  {\n                "lead_data": {\n                    "name": "João Moura",\n                    "job_title": "Director of Engineering",\n                    "company": "Clearbit",\n                    "email": "joao@clearbit.com",\n                    "use_case": "Using AI Agent to do better data enrichment."\n                },\n            }\n\ntest_crew_1 = Crew(\n  agents=[lead_data_agent],\n  tasks=[lead_data_task] )\n\nsample_result_1 = test_crew_1.kickoff(sample_input)\n'

In [31]:
"""
test_crew_2 = Crew(
  agents=[lead_data_agent,cultural_fit_agent],
  tasks=[lead_data_task,cultural_fit_task] )

test_crew_2 = test_crew_2.kickoff(sample_input)"""

'\ntest_crew_2 = Crew(\n  agents=[lead_data_agent,cultural_fit_agent],\n  tasks=[lead_data_task,cultural_fit_task] )\n\ntest_crew_2 = test_crew_2.kickoff(sample_input)'

In [32]:
"""
test_crew_3 = Crew(
  agents=[lead_data_agent,cultural_fit_agent,scoring_validation_agent],
  tasks=[lead_data_task,cultural_fit_task,scoring_validation_task],
   verbose=True )

test_crew_3 = test_crew_3.kickoff(sample_input)"
"""

'\ntest_crew_3 = Crew(\n  agents=[lead_data_agent,cultural_fit_agent,scoring_validation_agent],\n  tasks=[lead_data_task,cultural_fit_task,scoring_validation_task],\n   verbose=True )\n\ntest_crew_3 = test_crew_3.kickoff(sample_input)"\n'

In [33]:
"""
import json
fist_pipeline_data = json.loads(test_crew_3.raw)
print(test_crew_3.raw)
"""


'\nimport json\nfist_pipeline_data = json.loads(test_crew_3.raw)\nprint(test_crew_3.raw)\n'

## Email Engagement Crew

In [34]:
# Creating Agents
email_content_specialist = Agent(
  config=email_agents_config['email_content_specialist'],
  #llm=ollama_llm
)

engagement_strategist = Agent(
  config=email_agents_config['engagement_strategist'],
  #llm=ollama_llm
)

# Creating Tasks
email_drafting = Task(
  config=email_tasks_config['email_drafting'],
  agent=email_content_specialist
)

engagement_optimization = Task(
  config=email_tasks_config['engagement_optimization'],
  agent=engagement_strategist
)


In [35]:
# Creating Crew
email_writing_crew = Crew(
  agents=[
    email_content_specialist,
    engagement_strategist
  ],
  tasks=[
    email_drafting,
    engagement_optimization
  ],
  verbose=True
)

2025-03-06 15:15:08,160 - 8687798336 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed


In [36]:
"""

sample_result_2 = email_writing_crew.kickoff(fist_pipeline_data)
"""

'\n\nsample_result_2 = email_writing_crew.kickoff(fist_pipeline_data)\n'

In [37]:
"""
leads = {
            "lead_data": {
                "name": "João Moura",
                "job_title": "Director of Engineering",
                "company": "Clearbit",
                "email": "joao@clearbit.com",
                "use_case": "Using AI Agent to do better data enrichment."
            },
        }
        
    
scores = lead_scoring_crew.kickoff(leads)
"""

'\nleads = {\n            "lead_data": {\n                "name": "João Moura",\n                "job_title": "Director of Engineering",\n                "company": "Clearbit",\n                "email": "joao@clearbit.com",\n                "use_case": "Using AI Agent to do better data enrichment."\n            },\n        }\n        \n    \nscores = lead_scoring_crew.kickoff(leads)\n'

## Creating Complete Sales Flow

In [76]:
from crewai import Flow
import json
from crewai.flow.flow import listen, start

class SalesPipeline(Flow):
    @start()
    def fetch_leads(self):
        # Pull our leads from the database
        leads =[ {
                "lead_data": {
                    "name": "João Moura",
                    "job_title": "Director of Engineering",
                    "company": "Clearbit",
                    "email": "joao@clearbit.com",
                    "use_case": "Using AI Agent to do better data enrichment."
                },
            },
            {
                "lead_data": {
                    "name": "Thar Htet San",
                    "job_title": "Senior AI Engineer",
                    "company": "ONOW",
                    "email": "tharhtetsan.ai@gmail.com",
                    "use_case": "Using AI Agent to solve real world problem."
                },
            }

        ]
        
        return leads

    @listen(fetch_leads)
    def score_leads(self, leads):
        scores = []
        for cur_lead in leads:
            cur_score = lead_scoring_crew.kickoff(cur_lead)
            scores.append(json.loads(cur_score.raw))
        self.state["score_crews_results"] = scores
        return scores

    @listen(score_leads)
    def store_leads_score(self, scores):
        # Here we would store the scores in the database
        return scores

    @listen(score_leads)
    def filter_leads(self, scores):
        print(scores)
        print("###############filter_leads##################")
        for cur_score in scores:
            socres_dict = cur_score
            socre_marks = socres_dict['lead_score']['score']
            print("socres_dict : ",socres_dict)
            print("socre_marks : ",socre_marks)
        print("#################################")
        
        return [score for score in scores if score['lead_score']['score'] > 75]

    @listen(filter_leads)
    def write_email(self, leads):
        print("###############write_email##################")
        for lead in leads:
         print(lead)
         print("#################################")
        
        print("#################################")
        
        emails = email_writing_crew.kickoff_for_each(leads)
        return emails

    @listen(write_email)
    def send_email(self, emails):
        # Here we would send the emails to the leads
        return emails

flow = SalesPipeline()

In [90]:
## Plotting the flow
flow.plot()

Plot saved as crewai_flow.html


In [91]:
from IPython.display import IFrame

IFrame(src='./crewai_flow.html', width='150%', height=600)

In [79]:
#flow kickoff




emails = await flow.kickoff_async()

# Agent: Lead Data Specialist
## Task: Collect and analyze the following information about the lead:
- Personal Information:
  - Name: Obtain the full name of the lead.
  - Job Title: Determine the lead's current job title.
  - Role Relevance: Assess how relevant the lead's role is to the decision-making process on a scale from 0 to 10.
  - Professional Background: Optionally, gather a brief description of the lead's professional background.

- Company Information:
  - Company Name: Identify the name of the company the lead works for.
  - Industry: Determine the industry in which the company operates.
  - Company Size: Estimate the size of the company in terms of employee count.
  - Revenue: If available, collect information on the annual revenue of the company.
  - Market Presence: Evaluate the company's market presence on a scale from 0 to 10.

- Our Company and Product:
  - Company Name: CrewAI
  - Product: Multi-Agent Orchestration Platform
  - ICP: Enterprise companies looking int

2025-03-06 15:27:46,965 - 8687798336 - __init__.py-__init__:537 - WARNING: Overriding of current TracerProvider is not allowed




# Agent: Lead Scorer and Validator
## Final Answer: 
{
  "personal_info": {
    "name": "Thar Htet San",
    "job_title": "Senior AI Engineer",
    "role_relevance": 8,
    "professional_background": null
  },
  "company_info": {
    "company_name": "ONOW Ascent Inc",
    "industry": "Technology, Information and Internet",
    "company_size": 10,
    "revenue": null,
    "market_presence": 6
  },
  "lead_score": {
    "score": 74,
    "scoring_criteria": [
      "Role Relevance: 8",
      "Company Size: 10",
      "Market Presence: 6",
      "Cultural Fit: 8"
    ],
    "validation_notes": "The lead score is high because of Thar's strong relevance in AI technology and the alignment of both organizations' missions and values."
  }
}


[{'personal_info': {'name': 'João Moura', 'job_title': 'Director of Engineering', 'role_relevance': 8, 'professional_background': 'João Moura has a robust background in engineering leadership, mainly focusing on data and automation technologies which ali

In [82]:
emails

[CrewOutput(raw='João, with your expertise at Clearbit, you know the competitive edge that Agentic automation offers. Let’s schedule a quick 15-minute call to discuss how CrewAI’s Multi-Agent Orchestration Platform can tailor AI Agents to elevate your productivity software. Click here to pick a time that works for you: [Schedule a Call](#). The potential for efficiency and innovation is immense. If you want to see our platform in action first, reply to this email and I can send a personalized demo just for Clearbit. Don’t miss out on the chance to transform your processes!', pydantic=None, json_dict=None, tasks_output=[TaskOutput(description='Craft a highly personalized email using the lead\'s name, job title, company information, and any relevant personal or company achievements. The email should speak directly to the lead\'s interests and the needs of their company. This is not as cold outreach as it is a follow up to a lead form, so keep it short and to the point. Don\'t use any sal

In [83]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([flow.state["score_crews_results"][0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

AttributeError: 'dict' object has no attribute 'token_usage'

In [84]:
print(emails[0].raw)

João, with your expertise at Clearbit, you know the competitive edge that Agentic automation offers. Let’s schedule a quick 15-minute call to discuss how CrewAI’s Multi-Agent Orchestration Platform can tailor AI Agents to elevate your productivity software. Click here to pick a time that works for you: [Schedule a Call](#). The potential for efficiency and innovation is immense. If you want to see our platform in action first, reply to this email and I can send a personalized demo just for Clearbit. Don’t miss out on the chance to transform your processes!


In [85]:
print(emails[1].raw)

IndexError: list index out of range

In [86]:
import pandas as pd

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])

# Calculate total costs
costs = 0.150 * df_usage_metrics['total_tokens'].sum() / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Display the DataFrame
df_usage_metrics

Total costs: $0.0002


/var/folders/xd/3z5vvpds0zxf_pypxd4cn3d80000gn/T/ipykernel_51521/99626748.py:4: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  df_usage_metrics = pd.DataFrame([emails[0].token_usage.dict()])


,total_tokens,prompt_tokens,cached_prompt_tokens,completion_tokens,successful_requests
0,1250,1012,0,238,2


In [87]:
## plotting the flow again
flow = SalesPipeline()
flow.plot()

Plot saved as crewai_flow.html


In [89]:
from IPython.display import IFrame

IFrame(src='./crewai_flow_complex.html', width='150%', height=600)